In [2]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/40.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/40.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/40.2 MB ? eta -:--:--
    --------------------------------------- 0.5/40.2 MB 1.0 MB/s eta 0:00:38
    --------------------------------------- 0.8/40.2 MB 1.2 MB/s eta 0:00:33
    --------------------------------------- 0.8/40.2 MB 1.2 MB/s eta 0:00:33
   - -------------------------------------- 1.0/40.2 MB 1.1 MB/s eta 0:00:36
   - -------------------------------------- 1.3/40.2 MB 972.7 kB/s eta 0:00:40
   - -------------------------------------- 1.6/40.2 MB 1.1 MB/s eta 0:00:37
   - -------------------------------------- 1.8/40.2 MB 1.1 MB/s eta 0:00:36
   -- ------------------------------------- 2.1/40.2 MB 1.1 MB/s eta 0:00:34
   -- ------------------------------------- 2.4/40.2 MB 1.1 MB/s eta 0:00:35
   -- -----------

In [3]:
import os
import cv2
import numpy as np
from glob import glob
import random

In [4]:
input_folder = "Images"      # Your folder with 21 images
output_folder = "hazy_images"      # Output folder

os.makedirs(output_folder, exist_ok=True)

In [8]:
def add_haze(img, beta=2.0, A=None):
    img = img.astype(np.float32) / 255.0
    h, w, _ = img.shape

    # Stronger depth map (more variation)
    gradient = np.tile(np.linspace(0, 1.5, h), (w,1)).T
    noise = cv2.GaussianBlur(np.random.rand(h, w), (51,51), 0)
    depth = 0.6 * gradient + 0.4 * noise

    # Higher beta = denser haze
    t = np.exp(-beta * depth)
    t = np.expand_dims(t, axis=2)

    # Strong atmospheric light (slightly warm smog tone)
    if A is None:
        A = np.array([0.95, 0.9, 0.85])  # yellowish dense haze

    hazy = img * t + A * (1 - t)
    hazy = np.clip(hazy, 0, 1)

    return (hazy * 255).astype(np.uint8)

In [9]:
def random_crop(img, crop_scale=0.8):
    h, w, _ = img.shape
    new_h = int(h * crop_scale)
    new_w = int(w * crop_scale)

    y = random.randint(0, h - new_h)
    x = random.randint(0, w - new_w)

    cropped = img[y:y+new_h, x:x+new_w]
    return cv2.resize(cropped, (w, h))

In [10]:
image_paths = glob(os.path.join(input_folder, "*"))

target_count = 75
current_count = 0

while current_count < target_count:
    for path in image_paths:
        if current_count >= target_count:
            break

        img = cv2.imread(path)

        if img is None:
            continue

        # Random break (crop)
        if random.random() > 0.5:
            img = random_crop(img, crop_scale=random.uniform(0.6, 0.9))

        # Random flip
        if random.random() > 0.5:
            img = cv2.flip(img, 1)

        # Random haze density
        beta = random.uniform(1.8,3.0)

        hazy_img = add_haze(img, beta=beta)

        filename = f"hazy_{current_count+1}.jpg"
        cv2.imwrite(os.path.join(output_folder, filename), hazy_img)

        current_count += 1

print("Done! Generated", current_count, "hazy images.")

Done! Generated 75 hazy images.
